---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [20]:
def answer_one():
    G = nx.read_edgelist('Employee_Movie_Choices.txt',delimiter = '\t') 
    return G
# plot_graph(answer_one())

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [52]:
def answer_two(): 
    
    G = answer_one()
    for node in G.nodes():
        if node in employees:
            G.add_node(node, type="employee")
        elif node in movies:
            G.add_node(node, type="movie")
            
    return G
# answer_two().nodes(data = True)

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [145]:
def answer_three():
    G = answer_two()    
    return bipartite.weighted_projected_graph(G, employees)

# plot_graph(answer_three(),weight_name='weight')

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [217]:
def answer_four():
    
    df_g = pd.DataFrame(answer_three().edges(data=True), columns=['employee1', 'employee2', 'attributes'])
    df_g['weight'] = df_g['attributes'].apply(lambda x: x['weight'])
    
    # "duplicating" data in order to have both combinations of employee names
    df_g_2 = df_g.rename(columns={"employee1":"employee2_","employee2":"employee1_"})
    df_g_2.rename(columns={"employee1_":"employee1","employee1_":"employee2"}, inplace=True)
    
    # consolidating dataframes
    df_cons = pd.concat([df_g,df_g_2])
    df_cons['employee1'] = df_cons.apply(lambda x : x['employee2'] if str(x['employee1'])   == 'nan' else x['employee1'] , axis = 1)
    df_cons['employee2'] = df_cons.apply(lambda x : x['employee2_'] if str(x['employee2_']) != 'nan' else x['employee2'] , axis = 1)
    weights_cons = df_cons[['employee1','employee2','weight']]
    
    #adding information to score data
    rel_score = pd.read_csv('Employee_Relationships.txt',delimiter = '\t',names = ['employee1', 'employee2', 'score'])
    rel_score = rel_score.merge(weights_cons, left_on = ['employee1','employee2'], right_on = ['employee1','employee2'], how='left').fillna(0)
    
    return rel_score['score'].corr(rel_score['weight'])
answer_four()

0.78839622217334748